# Revised case normalization for KSSG 2021 

Notes: 

- All cases (25 of 25) could be matched between the revised cases from DtoD and the BFS DB data of KSSG 2021.


In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:

file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/KSSG_2021.xlsx'),
         'KSSG HQ',
         '2021',
         'Änderungen_KSSG_2021')

print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/KSSG_2021.xlsx', hospital_name_db='KSSG HQ', year='2021', sheet='Änderungen_KSSG_2021')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-10 11:38:51.638 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 25 cases for KSSG HQ 2021
2022-11-10 11:38:51.641 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:59 - TYPES:
tranche                   string
datum lieferung           string
case_id                   object
patient_id                object
kkik                      string
gender                    string
age_years                  int64
bfs_code                  string
duration_of_stay           int64
pflegetage neu            string
pccl                       int64
pccl neu                  string
old_pd                    string
primary_diagnosis         string
added_icds                string
removed_icds              string
added_chops               string
removed_chops             string
drg                       string
drg neu                   string
cw alt                    string
cw neu                    string
cw-änderung 

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,1112425128,F1B1A46BC1FEF112,M,56,5,1112425128,K831,K8031,M100,3,H41C,"[K8510, D6959]",[],[],[]
1,1112427844,59106C27B3536BE5,M,64,9,1112427844,R5088,R5088,M100,3,T62C,"[I691, G819]",[],[],[]
2,1112441515,BB0033F7EF9CCB1F,W,70,3,1112441515,C833,C833,M100,3,R61D,[D6110],[],[],[]
3,1112445586,73EFDADADF92D6A4,W,93,7,1112445586,I5001,I5001,M100,2,F62D,[F050],[],[],[]
4,1112426216,FD14EA356F1A32E4,M,78,4,1112426216,J128,J128,M100,2,E77E,[J4412],[J4482],[],[]
5,1112427071,C47C1FF3CA495488,W,70,2,1112427071,M6286,M6286,M100,2,I75C,[F132],[],[],[]
6,1112430217,1E6CD6DFCCD600D9,M,87,9,1112430217,J128,J128,M100,3,E77D,[E440],[E46],[890A32],[]
7,1112432613,C997F68564CDE89B,M,51,10,1112432613,J128,J128,M100,2,E77E,[D803],[],[],[]
8,1112438704,5753DCD32E7CF5CA,W,71,5,1112438704,I634,I634,M100,3,B70D,[G836],[G510],[],[]
9,1112448747,3A15CC21343AFF78,M,64,2,1112448747,K912,K912,M100,3,G48C,[L8914],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-10 11:38:56.705 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 28977 rows from the DB, for the hospital 'KSSG HQ' in 2021


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,402682,1112497209,C437,"[G402, I693, F018, C774]",86431C:R:20210604,"[866B2C:R:20210604, 402411:R:20210604, 401920:R:20210604, 8688I5:R:20210604, 935713::20210604, ...]",W,85,0,0,6,0,01,2021-06-04,0,00,2021-06-10
1,402895,1112438704,I634,"[G832, R471, I1310, I480, Z867, ...]",99B711::20210131,"[99BA13::20210131, 883850::20210131, 870411::20210131, 870499::20210131, 874199::20210131, ...]",W,71,0,0,5,0,01,2021-01-31,0,00,2021-02-05
2,405303,1112426216,J128,"[U071, I1190, I2519, E1172, N083, ...]",998425::20210104,[],M,78,0,0,4,0,01,2021-01-04,0,00,2021-01-08
3,405413,1112465409,J180,"[U990, I1090, K766, R64, E063, ...]",874199::20210326,[],W,74,0,0,5,0,01,2021-03-26,0,00,2021-03-31
4,405419,1112425128,K8031,"[Y579, F112, Z5183, F192, K703, ...]",5188::20201231,[890A32::20210104],M,56,0,0,5,0,01,2020-12-31,0,00,2021-01-05


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-10 11:38:57.311 | INFO     | src.revised_case_normalization.notebook_functions.group:group:83 - Grouping 25 cases ...
2022-11-10 11:38:58.257 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:88 - Grouped 25 cases into: 25 revisions, 267 diagnoses rows, 104 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,402682,J08A,2.16,2.16,4,2022-12-31
1,402895,B70A,2.27,2.27,4,2022-12-31
2,405303,E77D,0.83,0.83,3,2022-12-31
3,405413,E77C,1.10,1.10,4,2022-12-31
4,405419,H41B,2.25,2.25,4,2022-12-31
5,408828,R61C,1.51,1.51,4,2022-12-31
6,408918,F62C,1.00,1.00,3,2022-12-31
7,409053,I72Z,1.07,1.07,3,2022-12-31
8,409471,D02B,6.02,6.02,4,2022-12-31
9,411569,I27A,3.34,3.34,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-10 11:38:58.276 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:249 - Trying to insert 25 cases into the 'Revisions' table ...
2022-11-10 11:38:59.201 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:286 - Inserted 25 cases into the 'Revisions' table
2022-11-10 11:38:59.204 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:298 - Trying to insert 267 rows into the 'Diagnoses' table ...
2022-11-10 11:39:00.970 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:336 - Inserted 267 rows into the 'Diagnoses' table
2022-11-10 11:39:00.974 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:347 - Trying to insert 104 rows into the 'Procedures' table ...
2022-11-10 11:39:02.299 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:393 - Inserted 104 rows into the 'Procedures' table
